In [1]:
import json

# Load the lyrics.json file
with open('./utils/lyrics.json') as json_file:
    lyrics = json.load(json_file)

In [6]:
# Display the first 5 elements
print(lyrics['Crash Into Me'].keys())

dict_keys(['lyrics', 'title', 'artist', 'album', 'source', 'music', 'description'])


In [7]:
print(lyrics.keys())    

dict_keys(['Crash Into Me', 'Ants Marching', '#41', 'Two Step', 'Grey Street', 'Warehouse', 'Samurai Cop (Oh Joy Begin)', 'Satellite', 'The Space Between', 'Tripping Billies', 'Jimi Thing', 'Crush', 'You and Me', 'Granny', 'Say Goodbye', 'The Stone', 'Grace Is Gone', 'Christmas Song', 'Dancing Nancies', 'Bartender', 'Lie in Our Graves', 'Cry Freedom', 'What Would You Say', 'Come On Come On', 'The Song That Jane Likes'])


In [2]:
import json
from datasets import Dataset
import numpy as np
import os

# Load song metadata from JSON
with open('./utils/lyrics.json', 'r') as f:
    song_metadata = json.load(f)

song_name_list = ['Crash Into Me', 'Ants Marching', '#41', 'Two Step',
                'Grey Street', 'Warehouse', 'Samurai Cop (Oh Joy Begin)',
                'Satellite', 'The Space Between', 'Tripping Billies',
                'Jimi Thing', 'Crush', 'You and Me']

# If the keys are not in the song_name_list, remove them
for key in list(song_metadata.keys()):
    if key not in song_name_list:
        del song_metadata[key]



In [5]:
import librosa
from scipy.io.wavfile import write

# Create the directory if it doesn't exist
if not os.path.exists('./data/dmb_audio'):
    os.makedirs('./data/dmb_audio')

# Function to load and segment audio
def load_and_segment_audio(file_path, song_title, segment_length=15):
    y, sr = librosa.load(file_path, sr=None)
    audio_length = librosa.get_duration(y=y, sr=sr)
    num_segments = int(audio_length // segment_length)
    remainder = audio_length % segment_length
    segments = []
    metadata = []

    for i in range(num_segments):
        start = int(i * segment_length * sr)
        end = int((i + 1) * segment_length * sr)
        segments.append(y[start:end])
        metadata.append({'segment_num': i, 'is_padded': False})
    
    if remainder > 0:
        start = int(num_segments * segment_length * sr)
        end = int((num_segments + 1) * segment_length * sr)
        padded_segment = np.pad(y[start:], (0, end - start - len(y[start:])))
        segments.append(padded_segment)
        metadata.append({'segment_num': num_segments, 'is_padded': True})

    for i, segment in enumerate(segments):
        new_file_path = f"./data/dmb_audio/{song_title.replace(' ', '_')}_{i}.wav"
        write(new_file_path, sr, (segment * 32767).astype(np.int16))  # Convert float to 16-bit PCM format
        metadata[i]['file_name'] = new_file_path

    return segments, metadata

# Initialize empty lists to hold all segments and metadata across multiple songs
all_segments = []
# Initialize empty list to hold metadata for all segments across multiple songs
all_metadata = []

# Iterate over each song's metadata and corresponding audio file
for song_title, metadata in song_metadata.items():
    if song_title != 'Crash Into Me':
        audio_file = f"../dmb_audio/{song_title}.wav"  # Replace with the actual file path
        segments, segment_metadata = load_and_segment_audio(audio_file, song_title)

        for i, seg_meta in enumerate(segment_metadata):
            # Combine song-specific metadata with segment-specific metadata
            combined_metadata = {
                'lyrics': metadata.get('lyrics', None),
                'title': metadata.get('title', None),
                'artist': metadata.get('artist', None),
                'album': metadata.get('album', None),
                'source': metadata.get('source', None),
                'description': metadata.get('description', None),
                'segment_num': seg_meta['segment_num'],
                'is_padded': seg_meta['is_padded'],
                'file_name': seg_meta['file_name']
                #'music': segments[i]  # Assuming you want to include the segment itself
            }
            all_metadata.append(combined_metadata)
    else:
        pass
# Create Hugging Face Dataset
dataset = Dataset.from_dict({
    'metadata': all_metadata
})

# Save the dataset
dataset.save_to_disk('./utils/dataset/')  # Replace with your desired path


Saving the dataset (0/1 shards):   0%|          | 0/265 [00:00<?, ? examples/s]

In [20]:
# Create a .jsonl file for the metadata
import json
for data in dataset["metadata"]:
    with open('./utils/dataset/metadata2.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

In [30]:
from datasets import Audio
audio_dataset = dataset.cast_column("file_name", Audio())

ValueError: Couldn't cast
metadata: struct<album: string, artist: string, description: string, file_name: string, is_padded: bool, lyrics: string, segment_num: int64, source: string, title: string>
  child 0, album: string
  child 1, artist: string
  child 2, description: string
  child 3, file_name: string
  child 4, is_padded: bool
  child 5, lyrics: string
  child 6, segment_num: int64
  child 7, source: string
  child 8, title: string
-- schema metadata --
huggingface: '{"info": {"features": {"metadata": {"album": {"dtype": "str' + 431
to
{'metadata': {'album': Value(dtype='string', id=None), 'artist': Value(dtype='string', id=None), 'description': Value(dtype='string', id=None), 'file_name': Value(dtype='string', id=None), 'is_padded': Value(dtype='bool', id=None), 'lyrics': Value(dtype='string', id=None), 'segment_num': Value(dtype='int64', id=None), 'source': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None)}, 'file_name': Audio(sampling_rate=None, mono=Tru